In [1]:
import os
import csv
from collections import defaultdict
import json

# To finetune

In [44]:
DATASET_VAR = 'orig'
DATASET_NAME = 'dataset'
RESULTS_PATH = f'/home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/{DATASET_VAR}'
VALID_TABLE = f'{RESULTS_PATH}/valid/BirdNET_SelectionTable.txt'
TRAIN_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/train'
VALID_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/valid'
TEST_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/test'
TEST_TABLE = f'{RESULTS_PATH}/test/BirdNET_SelectionTable.txt'

In [45]:
print(f'python -m birdnet_analyzer.train --i {TRAIN_PATH} --o {RESULTS_PATH}/{DATASET_VAR}.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file {RESULTS_PATH}/{DATASET_VAR}.npz')

python -m birdnet_analyzer.train --i /home/giacomoschiavo/segments/dataset/train --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/orig/orig.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/orig/orig.npz


# To analize

In [46]:
MIN_CONF = '0.05'

In [47]:
print(f'python -m birdnet_analyzer.analyze --i {VALID_PATH} --o {RESULTS_PATH}/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/valid --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/orig/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/orig/orig.tflite


In [48]:
print(f'python -m birdnet_analyzer.analyze --i {TEST_PATH} --o {RESULTS_PATH}/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/test --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/orig/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/orig/orig.tflite


# Analysis

In [49]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/species_dict_map.json') as f:
    species_dict = json.load(f)
inv_species_dict = {value: key for key, value in species_dict.items()}

In [50]:
def get_conf_scores():
    conf_scores = {}

    #  {'Aeroplane': [(np.float32(0.0), False),
    #               (np.float32(1.3937646e-30), False),
    #               (np.float32(1.0654355e-25), False),
    #               (np.float32(0.0), False),

    with open(VALID_TABLE, 'r') as f:
        reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
        for row in reader:
            file_path = row['Begin Path']
            pred_species_name = row['Common Name']
            if row['Common Name'] in inv_species_dict:
                pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
            true_species_name = file_path.split('/')[-2]
            confidence = float(row['Confidence'])
            if pred_species_name not in conf_scores:
                conf_scores[pred_species_name] = []
            is_correct = pred_species_name == true_species_name
            conf_scores[pred_species_name].append((confidence, is_correct))
    return conf_scores
            

In [51]:
conf_scores = get_conf_scores()

In [52]:
from sklearn.metrics import f1_score
import numpy as np 

DEFAULT_THRESH = 0.25

def compute_best_thresholds(conf_scores, num_thresholds=200, min_thresh=0.001, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = DEFAULT_THRESH
        best_f1 = 0.0

        for thresh in np.linspace(min_thresh, max_thresh, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 {species} -> {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [53]:
best_thresholds = compute_best_thresholds(conf_scores)

📊 Phylloscopus collybita_Common Chiffchaff -> 0.073, F1-score: 0.526
📊 Sylvia atricapilla_Eurasian Blackcap -> 0.250, F1-score: 0.000
📊 Anthus trivialis_Tree Pipit -> 0.950, F1-score: 0.437
📊 Fringilla coelebs_Common Chaffinch -> 0.053, F1-score: 0.852
📊 Certhia familiaris_Eurasian Treecreeper -> 0.325, F1-score: 0.771
📊 Regulus regulus_Goldcrest -> 0.087, F1-score: 0.845
📊 Muscicapa striata_Spotted Flycatcher -> 0.250, F1-score: 0.000
📊 Coccothraustes coccothraustes_Hawfinch -> 0.912, F1-score: 0.947
📊 Erithacus rubecula_European Robin -> 0.220, F1-score: 0.439
📊 Periparus ater_Coal Tit -> 0.950, F1-score: 0.693
📊 Regulus ignicapilla_Common Firecrest -> 0.306, F1-score: 0.615
📊 Troglodytes troglodytes_Eurasian Wren -> 0.096, F1-score: 0.910
📊 Parus major_Great Tit -> 0.058, F1-score: 0.200
📊 Turdus merula_Eurasian Blackbird -> 0.125, F1-score: 0.286
📊 Lophophanes cristatus_Crested Tit -> 0.263, F1-score: 0.857
📊 Cuculus canorus_Common Cuckoo -> 0.250, F1-score: 0.000


In [54]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_test.json') as f:
    true_segments_test = json.load(f)

In [55]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_train.json') as f:
    true_segments_full = json.load(f)

In [56]:
true_segments_full.update(true_segments_test)

In [57]:
pred_segments = {}
#  {'Aeroplane': [(np.float32(0.0), False),
#               (np.float32(1.3937646e-30), False),
#               (np.float32(1.0654355e-25), False),
#               (np.float32(0.0), False),

with open(TEST_TABLE, 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
    for row in reader:
        file_path = row['Begin Path']
        audio_name = os.path.basename(file_path)    # in test non sono presenti WABAD o augm
        only_audio_name = "_".join(audio_name.split("_")[:2]) + ".WAV"
        segm = "_".join(audio_name.split(".")[0].split("_")[-2:])
        pred_species_name = row['Common Name']
        if row['Common Name'] in inv_species_dict:
            pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
        # if len(file_path.split('/')[-2].split('_')) == 1:       # skip if None, Pecking, etc. 
        #     continue
        confidence = float(row['Confidence'])
        if only_audio_name not in pred_segments:
            pred_segments[only_audio_name] = {}
        if pred_species_name not in best_thresholds:
            best_thresholds[pred_species_name] = DEFAULT_THRESH
        if confidence >= best_thresholds[pred_species_name]:
            if segm not in pred_segments[only_audio_name]:
                pred_segments[only_audio_name][segm] = set()
            if "None" in pred_segments[only_audio_name][segm]:
                continue
            if pred_species_name == "None":
                pred_segments[only_audio_name][segm] = set(["None"])
            else:
                pred_segments[only_audio_name][segm].add(pred_species_name)
            

In [58]:
from sklearn.preprocessing import MultiLabelBinarizer

test_species_list = os.listdir(TEST_PATH)
# test_species_list = [species for species in test_species_list if len(species.split('_')) > 1]
mlb = MultiLabelBinarizer()
mlb.fit([test_species_list])

len(mlb.classes_)

21

In [59]:
true_segments = defaultdict(dict)

for species in os.listdir(TEST_PATH):
    if species not in test_species_list:
        print(species, 'ignored')
        continue
    for audio in os.listdir(os.path.join(TEST_PATH, species)):
        audio = audio.split('.')[0]
        date, time, segm1, segm2 = audio.split('_')
        audio_name = '_'.join([date, time]) + '.WAV'
        segm = '_'.join([segm1, segm2])
        if segm not in true_segments[audio_name]:
            true_segments[audio_name][segm] = []
        true_segments[audio_name][segm].extend([species])

In [60]:
for audio in true_segments.keys():
    if audio not in pred_segments:
        pred_segments[audio] = {}
    for segm in true_segments[audio].keys():
        if segm not in pred_segments[audio]:
            pred_segments[audio][segm] = []
        

In [61]:
y_pred = []
y_true = []
for audio in pred_segments:
    # sort in increasing order
    sortable_true_segments = { str(key): value for key, value in true_segments[audio].items() }
    sortable_pred_segments = { str(key): value for key, value in pred_segments[audio].items() }
    sorted_true_segments = dict(sorted(sortable_true_segments.items()))
    sorted_pred_segments = dict(sorted(sortable_pred_segments.items()))
    y_true.append(mlb.transform(sorted_true_segments.values()))     # apply transform on every label of every segment
    y_pred.append(mlb.transform(sorted_pred_segments.values()))     # apply transform on every label of every segment

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

In [62]:
from sklearn.metrics import classification_report
import pandas as pd
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Aeroplane,0.000000,0.000000,0.000000,22.0
Anthus trivialis_Tree Pipit,0.826923,0.537500,0.651515,160.0
Certhia familiaris_Eurasian Treecreeper,0.776316,0.641304,0.702381,92.0
Coccothraustes coccothraustes_Hawfinch,0.826087,0.218391,0.345455,87.0
Cuculus canorus_Common Cuckoo,0.000000,0.000000,0.000000,3.0
Erithacus rubecula_European Robin,0.873950,0.374101,0.523929,556.0
Fringilla coelebs_Common Chaffinch,0.862210,0.461314,0.601046,1370.0
Lophophanes cristatus_Crested Tit,1.000000,0.304348,0.466667,23.0
Loxia curvirostra_Common Crossbill,0.000000,0.000000,0.000000,36.0
Muscicapa striata_Spotted Flycatcher,1.000000,0.005780,0.011494,173.0


In [63]:
with open(f'{RESULTS_PATH}/classification_report.json', 'w') as f:
    json.dump(report, f)

In [64]:
from sklearn.metrics import multilabel_confusion_matrix

mcm = multilabel_confusion_matrix(y_true, y_pred)

# Per visualizzarle in un DataFrame leggibile:
mlb_labels = mlb.classes_  # nome delle classi, già usato da te

# Costruisco un dataframe
conf_matrices = []
for idx, label in enumerate(mlb_labels):
    tn, fp, fn, tp = mcm[idx].ravel()
    conf_matrices.append({
        'Class': label,
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'TN': tn
    })

conf_df = pd.DataFrame(conf_matrices)
conf_df


,Class,TP,FP,FN,TN
0,Aeroplane,0,0,22,8953
1,Anthus trivialis_Tree Pipit,86,18,74,8797
2,Certhia familiaris_Eurasian Treecreeper,59,17,33,8866
3,Coccothraustes coccothraustes_Hawfinch,19,4,68,8884
4,Cuculus canorus_Common Cuckoo,0,0,3,8972
5,Erithacus rubecula_European Robin,208,30,348,8389
6,Fringilla coelebs_Common Chaffinch,632,101,738,7504
7,Lophophanes cristatus_Crested Tit,7,0,16,8952
8,Loxia curvirostra_Common Crossbill,0,0,36,8939
9,Muscicapa striata_Spotted Flycatcher,1,0,172,8802
